In [1]:
import numpy as np
import pyvista as pv

In [18]:
import torch

In [3]:
DXsec_Al = np.load('./DXsec_Al_1e8.npy')

In [2]:
import Multi_Species_Cu_Si_noAr

In [2]:
def slide2D_fractionZ(film, start, end, direction, fraction, value):
    if fraction == '+':
        if direction == 'y':
            slit = np.linspace(0, np.abs(end[2] - start[2]), np.abs(end[1] - start[1]))
            fraction = np.abs(int(slit[0]-slit[1]))
            print('y', slit)
            print('fraction', fraction)
            for i in range(np.abs(end[1] - start[1])):
                if end[1] > start[1]:
                    film[start[0]:end[0], start[1] + i, start[2]:start[2] + int(slit[i])] = value
                    for j in range(fraction):
                        film[start[0]:end[0], start[1] + i,start[2]+int(slit[i])+j] = 1/(fraction+1)*(fraction-j)
                elif end[1] < start[1]:
                    film[start[0]:end[0], start[1] - i, start[2]:start[2] + int(slit[i])] = value
                    for j in range(fraction):
                        film[start[0]:end[0], start[1] - i,start[2]+int(slit[i])+j] = 1/(fraction+1)*(fraction-j)
        elif direction == 'x':
            slit = np.linspace(0, np.abs(end[2] - start[2]), np.abs(end[0] - start[0]))
            fraction = np.abs(int(slit[0]-slit[1]))
            print('x', slit)
            print('fraction', fraction)
            for i in range(np.abs(end[2] - start[2])):
                if end[0] > start[0]:
                    film[start[0] + i, start[1]:end[1], start[2]:start[2] + int(slit[i])] = value
                    for j in range(fraction):
                        film[start[0] + i, start[1]:end[1], start[2]+int(slit[i])+j] = 1/(fraction+1)*(fraction-j)
                elif end[0] < start[0]:
                    film[start[0] - i, start[1]:end[1], start[2]:start[2] + int(slit[i])] = value
                    for j in range(fraction):
                        film[start[0] - i, start[1]:end[1], start[2]+int(slit[i])+j] = 1/(fraction+1)*(fraction-j)
    elif fraction == '-':
        if direction == 'y':
            slit = np.linspace(0, np.abs(end[2] - start[2]), np.abs(end[1] - start[1]))
            fraction = np.abs(int(slit[0]-slit[1]))
            print('y', slit)
            print('fraction', fraction)
            for i in range(np.abs(end[1] - start[1])):
                if end[1] > start[1]:
                    film[start[0]:end[0], start[1] + i, start[2] - int(slit[i]):start[2]+1] = value
                    for j in range(fraction):
                        film[start[0]:end[0], start[1] + i,start[2]-int(slit[i])-j] = 1/(fraction+1)*(fraction-j)
                elif end[1] < start[1]:
                    film[start[0]:end[0], start[1] - i, start[2] - int(slit[i]):start[2]+1] = value
                    for j in range(fraction):
                        film[start[0]:end[0], start[1] - i,start[2]-int(slit[i])-j] = 1/(fraction+1)*(fraction-j)
        elif direction == 'x':
            slit = np.linspace(0, np.abs(end[2] - start[2]), np.abs(end[0] - start[0]))
            fraction = np.abs(int(slit[0]-slit[1]))
            print('x', slit)
            print('fraction', fraction)
            for i in range(np.abs(end[2] - start[2])):
                if end[0] > start[0]:
                    film[start[0] + i, start[1]:end[1], start[2] - int(slit[i]):start[2]+1] = value
                    for j in range(fraction):
                        film[start[0] + i, start[1]:end[1], start[2] - int(slit[i]):start[2]] = 1/(fraction+1)*(fraction-j)
                elif end[0] < start[0]:
                    film[start[0] - i, start[1]:end[1], start[2] - int(slit[i]):start[2]+1] = value
                    for j in range(fraction):
                        film[start[0] - i, start[1]:end[1], start[2] - int(slit[i]):start[2]] = 1/(fraction+1)*(fraction-j)
    return film

In [ ]:
film = np.zeros((50, 200, 180))

bottom = 10

film = slide2D_fractionZ(film=film, start=[0, 129, 30], end=[50, 120, 10], direction='y', fraction='-', value=10)
film = slide2D_fractionZ(film=film, start=[0, 71, 30], end=[50, 80, 10], direction='y', fraction='-', value=10)

film[:, 80:121, 0:31] = 10

film[:, :, 0:bottom] = 10 # bottom

In [10]:
film = np.zeros((50, 200, 180))

bottom = 10

film[:, 80:121, 0:31] = 10

film[:, :, 0:bottom] = 10 # bottom

In [ ]:
depo1 = torch.Tensor(np.logical_and(film[:, :, :] !=0, film[:, :, :] < 10)).to_sparse()
depo1 = depo1.indices().numpy().T

substrute = torch.Tensor(film[:, :, :]==10).to_sparse()
substrute = substrute.indices().numpy().T
depomesh = pv.PolyData(depo1)
depomesh["radius"] = np.ones(depo1.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

# Progress bar is a new feature on master branch
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
# p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show_grid()
p.show()

In [11]:
etchfilm = np.zeros((50, 200, 180, 2))
etchfilm[:, :, :, 0] = film
etchfilm[:, :, :, 1] = film

In [4]:
fullCell=10

In [5]:
N = int(1e7)

In [6]:
import Multi_Species_bosch_etching_countByReaction_mirror

In [12]:
filmKDTree=np.array([[2, 1], [3, 1]])

In [13]:
react_table = np.array([[[0.900, 0, 1], [0.900, 0, 1]],
                        [[0.200, -1, 0], [0.075, 0, -1]]])

react_type_table = np.array([[2, 3],
                             [1, 0]])

In [14]:
Multi_Species_bosch_etching_countByReaction_mirror.react_table = react_table
Multi_Species_bosch_etching_countByReaction_mirror.react_type_table = react_type_table

In [15]:
logname = 'Multi_species_benchmark_1013_trench_test'
center = 100
density = 10
testEtch = Multi_Species_bosch_etching_countByReaction_mirror.etching(inputMethod='bunch', depo_or_etching='depo', 
                etchingPoint = np.array([25, center, bottom+50]),depoPoint = np.array([center, center, bottom+50]),
                density=density, center_with_direction=np.array([[25,center,75]]), 
                range3D=np.array([[0, 300, 0, 300, 0, 300]]), InOrOut=[1], yield_hist=np.array([None]),
                maskTop=40, maskBottom=98, maskStep=10, maskCenter=[25,center],mirrorGap=5,
                reaction_type=False, param = [1.6, -0.7],n=1,
                celllength=1e-5, kdtreeN=5, filmKDTree=filmKDTree,weight=0.2, tstep=1e-5,
                substrateTop=bottom,posGeneratorType='top', logname=logname)

In [16]:
celllength=1e-5
parcel = np.array([[center*celllength, center*celllength, 159*celllength, 0, 0, 1, center, center, 159, 0]])
step1 = testEtch.inputParticle(etchfilm, parcel, 'maxwell', 0, int(1e4),int(4.2e5),2, 10, 0)
# np.save('./bosch_data_1011_ratio08_trench_condition5_300wide/bosch_sf_test2', etchfilm)

(60, 210, 180)


particle input:  22%|██████████                                    | 22.0/100 [04:08<14:39, 11.3s/B]


KeyboardInterrupt: 

In [24]:
depo1 = torch.Tensor(np.logical_and(etchfilm[:, :, :,1] !=0, etchfilm[:, :, :,1] < 10)).to_sparse()
depo1 = depo1.indices().numpy().T

substrute = torch.Tensor(etchfilm[:, :, :,1]==10).to_sparse()
substrute = substrute.indices().numpy().T
depomesh = pv.PolyData(depo1)
depomesh["radius"] = np.ones(depo1.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

# Progress bar is a new feature on master branch
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
# p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show_grid()
p.show()

Widget(value='<iframe src="http://localhost:58680/index.html?ui=P_0x1f92fed9250_4&reconnect=auto" class="pyvis…

In [11]:
logname = 'Multi_species_benchmark_0729'
testEtch = Multi_Species_Cu_Si_noAr.etching(mirror=True,inputMethod='bunch', pressure_pa=0.001, temperature=300, chamberSize=etchfilm.shape,
                    depoThick=100, center_with_direction=np.array([[35,100,75]]), 
                    range3D=np.array([[0, 70, 0, 100, 0, 150]]), InOrOut=[1], yield_hist=np.array([None]),
                    reaction_type=False, param = [1.6, -0.7], N = N, 
                    sub_xy=[0,0], film=etchfilm, n=1, cellSize=etchfilm.shape, 
                    celllength=1e-5, kdtreeN=5, tstep=1e-5,
                    substrateTop=30,posGeneratorType='top', logname=logname)

In [ ]:
deposit = testEtch.depo_position_increase_cosVel_normal(125, N, 2e-2, 10)

In [ ]:
np.save('cvd_void_subDepo_thk110_tape115', deposit[0])

In [ ]:
deposit0 = np.load('./For_etching_transport_TS60_deposit_0604_4.npy')

In [ ]:
deposit[0][20, 117, 30:]

In [ ]:
sumFilm = np.sum(deposit[0], axis=-1)

depo1 = torch.Tensor(np.logical_and(film[:, :, :,]!=fullCell, sumFilm[:, :, :,]!=0)).to_sparse()
depo1 = depo1.indices().numpy().T

substrute = torch.Tensor(film[:, :, :,]==fullCell).to_sparse()
substrute = substrute.indices().numpy().T
depomesh = pv.PolyData(depo1)
depomesh["radius"] = np.ones(depo1.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

# Progress bar is a new feature on master branch
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

In [ ]:
deposit[2].shape

In [ ]:
point_cloud = pv.PolyData(deposit[1][:, 3:])
vectors = deposit[1][:, :3]

point_cloud['vectors'] = vectors
arrows = point_cloud.glyph(
    orient='vectors',
    scale=1000,
    factor=2,
)

# Display the arrows
plotter = pv.Plotter()
plotter.add_mesh(point_cloud, color='maroon', point_size=5.0, render_points_as_spheres=True)
# plotter.add_mesh(arrows, color='lightblue')
# plotter.add_point_labels([point_cloud.center,], ['Center',],
#                          point_color='yellow', point_size=20)
plotter.show_grid()
plotter.show()

In [ ]:
point_cloud = pv.PolyData(deposit[2][:, 3:])
vectors = deposit[2][:, :3]

point_cloud['vectors'] = vectors
arrows = point_cloud.glyph(
    orient='vectors',
    scale=1000,
    factor=2,
)

# Display the arrows
plotter = pv.Plotter()
plotter.add_mesh(point_cloud, color='maroon', point_size=5.0, render_points_as_spheres=True)
# plotter.add_mesh(arrows, color='lightblue')
# plotter.add_point_labels([point_cloud.center,], ['Center',],
#                          point_color='yellow', point_size=20)
plotter.show_grid()
plotter.show()

In [ ]:
deposit[0][20, 100, :]

In [ ]:
film = np.zeros((100, 100, 100, 2))

bottom = 10
film[:, :, 0:bottom, 0] = 10 # bottom

height = 80

film[:, :40, 0:height, 0] = 10
film[:, 60:, 0:height, 0] = 10
etchfilm = film


N = int(1e6)
velosity_matrix = np.zeros((N, 3))
tstep=1e-5
celllength=1e-5
# velosity_matrix[:, 0] = -1 * celllength /tstep
velosity_matrix[:, 1] = -1 * celllength /tstep
velosity_matrix[:, 2] = -1 * celllength /tstep

typeID = np.zeros(N)

print(velosity_matrix[0])

logname = 'Multi_species_benchmark_0729'
testEtch = Multi_Species_etching_depo.etching(mirror=True,inputMethod='bunch', pressure_pa=0.001, temperature=300, chamberSize=etchfilm.shape,
                    depoThick=90, center_with_direction=np.array([[35,100,75]]), 
                    range3D=np.array([[0, 70, 0, 100, 0, 150]]), InOrOut=[1], yield_hist=np.array([None]),
                    reaction_type=False, param = [1.6, -0.7], N = 300000, 
                    sub_xy=[0,0], film=etchfilm, n=1, cellSize=etchfilm.shape, 
                    celllength=1e-5, kdtreeN=5, tstep=1e-5,
                    substrateTop=40,posGeneratorType='benchmark',fullCell=10, logname=logname)


etching1 = testEtch.inputParticle(125, velosity_matrix, typeID, 2e-3, 20)

In [ ]:
etching1[0][39, 60, :]

In [ ]:
etching1[0][50, 60, :]

In [ ]:
sumFilm = np.sum(etching1[0], axis=-1)

depo1 = torch.Tensor(np.logical_and(sumFilm[:, :, :,]!=10, sumFilm[:, :, :,]!=0)).to_sparse()
depo1 = depo1.indices().numpy().T

substrute = torch.Tensor(sumFilm[:, :, :,]==10).to_sparse()
substrute = substrute.indices().numpy().T
depomesh = pv.PolyData(depo1)
depomesh["radius"] = np.ones(depo1.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

# Progress bar is a new feature on master branch
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

In [ ]:
point_cloud = pv.PolyData(etching1[1][:, 3:])
vectors = etching1[1][:, :3]

point_cloud['vectors'] = vectors
arrows = point_cloud.glyph(
    orient='vectors',
    scale=1000,
    factor=2,
)

# Display the arrows
plotter = pv.Plotter()
plotter.add_mesh(point_cloud, color='maroon', point_size=5.0, render_points_as_spheres=True)
# plotter.add_mesh(arrows, color='lightblue')
# plotter.add_point_labels([point_cloud.center,], ['Center',],
#                          point_color='yellow', point_size=20)
plotter.show_grid()
plotter.show()

In [ ]:
film = np.zeros((100, 100, 100, 2))

bottom = 10
fullcell = 100
film[:, :, 0:bottom, 0] = fullcell # bottom

height = 80

film[:, :40, 0:height, 0] = fullcell
film[:, 60:, 0:height, 0] = fullcell
etchfilm = film


N = int(1e6)
velosity_matrix = np.zeros((N, 3))
tstep=1e-5
celllength=1e-5
# velosity_matrix[:, 0] = -1 * celllength /tstep
velosity_matrix[:, 1] = -1 * celllength /tstep
velosity_matrix[:, 2] = -1 * celllength /tstep

typeID = np.zeros(N)

print(velosity_matrix[0])

logname = 'Multi_species_benchmark_0729'
testEtch = Multi_Species_etching_depo.etching(mirror=True,inputMethod='bunch', pressure_pa=0.001, temperature=300, chamberSize=etchfilm.shape,
                    depoThick=90, center_with_direction=np.array([[35,100,75]]), 
                    range3D=np.array([[0, 70, 0, 100, 0, 150]]), InOrOut=[1], yield_hist=np.array([None]),
                    reaction_type=False, param = [1.6, -0.7], N = 300000, 
                    sub_xy=[0,0], film=etchfilm, n=1, cellSize=etchfilm.shape, 
                    celllength=1e-5, kdtreeN=5, tstep=1e-5,
                    substrateTop=40,posGeneratorType='benchmark',fullCell=fullcell, logname=logname)


etching1 = testEtch.inputParticle(125, velosity_matrix, typeID, 2e-3, 20)

sumFilm = np.sum(etching1[0], axis=-1)

In [ ]:
depo1 = torch.Tensor(np.logical_and(sumFilm[:, :, :,]!=10, sumFilm[:, :, :,]!=0)).to_sparse()
depo1 = depo1.indices().numpy().T

substrute = torch.Tensor(sumFilm[:, :, :,]==10).to_sparse()
substrute = substrute.indices().numpy().T
depomesh = pv.PolyData(depo1)
depomesh["radius"] = np.ones(depo1.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

# Progress bar is a new feature on master branch
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

In [ ]:
etching1[0][45, 60, :]

In [ ]:
    # depo1 = torch.Tensor(np.logical_and(sumFilm[:, :, :,]!=10, sumFilm[:, :, :,]!=0)).to_sparse()
    # depo1 = depo1.indices().numpy().T

    # substrute = torch.Tensor(sumFilm[:, :, :,]==10).to_sparse()
    # substrute = substrute.indices().numpy().T
    # depomesh = pv.PolyData(depo1)
    # depomesh["radius"] = np.ones(depo1.shape[0])*0.5
    # geom = pv.Box()

    # submesh = pv.PolyData(substrute)
    # submesh["radius"] = np.ones(substrute.shape[0])*0.5

    # # Progress bar is a new feature on master branch
    # depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
    # subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

    # p = pv.Plotter()
    # p.add_mesh(depoglyphed, color='cyan')
    # p.add_mesh(subglyphed, color='dimgray')
    # p.enable_eye_dome_lighting()
    # p.show()

point_cloud = pv.PolyData(etching1[2][:, 3:])
vectors = etching1[2][:, :3]

point_cloud['vectors'] = vectors
arrows = point_cloud.glyph(
orient='vectors',
scale=1000,
factor=2,
)

# Display the arrows
plotter = pv.Plotter()
plotter.add_mesh(point_cloud, color='maroon', point_size=5.0, render_points_as_spheres=True)
# plotter.add_mesh(arrows, color='lightblue')
# plotter.add_point_labels([point_cloud.center,], ['Center',],
#                          point_color='yellow', point_size=20)
plotter.show_grid()
plotter.show()

In [ ]:
point_cloud = pv.PolyData(etching1[2][:, 3:])
vectors = etching1[2][:, :3]

point_cloud['vectors'] = vectors
arrows = point_cloud.glyph(
orient='vectors',
scale=1000,
factor=2,
)

# Display the arrows
plotter = pv.Plotter()
plotter.add_mesh(point_cloud, color='maroon', point_size=5.0, render_points_as_spheres=True)
# plotter.add_mesh(arrows, color='lightblue')
# plotter.add_point_labels([point_cloud.center,], ['Center',],
#                          point_color='yellow', point_size=20)
plotter.show_grid()
plotter.show()